# Transport Patients to Hospitals Using Search Methods
In this computer assignment, we implement 3 search algorithms such as BFS, IDS and A* to find a path for ambulance to transport patients to hospitals.

## Libraries
`time` is imported to measure time duration of solving the problem with each algorithm. Also `OrderedSet` is a data structure which not only access time is O(1) like set, but also remembers insertion order. 

In [1]:
# ! pip3 install ordered-set

In [2]:
import time
from ordered_set import OrderedSet
from IPython.display import Markdown, display

## Data
3 maps are given as testcases which includes '#', 'P', 'A' and numbers that means *wall*, *patient*, *ambulance* and the capacity of the hospital. To store patients, walls and hospitalls, *set*s are used to reduce access time.

In [3]:
def read_map(filename):
    file = list(open(filename, 'r'))
    walls_loc = set()
    patients_loc = set()
    hospitals_loc = set()
    for i in range(len(file)):
        for j in range(len(file[0])-1):
            if file[i][j] == 'A':
                ambulance_loc = [i, j]
            elif file[i][j] == '#':
                walls_loc.add((i, j))
            elif file[i][j] == 'P':   
                patients_loc.add((i, j))
            elif file[i][j].isdigit():
                hospitals_loc.add((i, j, int(file[i][j])))
    return ambulance_loc, walls_loc, patients_loc, hospitals_loc

## States
To implement search algorithms, it's necessary to store given map and sates in proper data structure. *State* is a class which stores ambulance, patients and hospitals location, g and h value that are used in A* algorithm. In order to use set to store *State*s, `__eq__` and `__hash__` methods sholud be overridden. Also, `get_next_states` is implemented to return feasible adjacent states. `eval_h` calculate heuristic of the state depending on the given method.

In [4]:
class State:
    def __init__(self, ambulance, patients, hospitals, path = "", g = 0, h = 0):
        self.ambulance = ambulance
        self.patients = patients
        self.hospitals = hospitals
        self.path = path
        self.g = g
        self.h = h
    def __eq__(self, other_state):
        if self.hospitals == other_state.hospitals and self.patients == other_state.patients and \
            self.ambulance == other_state.ambulance:
            return True
        else:
            return False
    def __hash__(self):
        return hash((tuple(self.ambulance), tuple(self.patients), tuple(self.hospitals)))
    def get_next_states(self, directions, walls_loc, max_hospital_size):
        next_states = []
        for key, value in directions.items():
            path = self.path + " " + key
            if (self.ambulance[0] + value[0], self.ambulance[1] + value[1]) in walls_loc:
                continue
            else:
                ambulance_loc = self.ambulance.copy()
                ambulance_loc[0] += value[0]; ambulance_loc[1] += value[1]
                patients_loc = self.patients.copy()
                hospitals_loc = self.hospitals.copy()
                new_g = self.g + 1
                if (self.ambulance[0] + value[0], self.ambulance[1] + value[1]) in self.patients:
                    if (self.ambulance[0] + 2*value[0], self.ambulance[1] + 2*value[1]) in walls_loc or \
                        (self.ambulance[0] + 2*value[0], self.ambulance[1] + 2*value[1]) in self.patients:
                        continue
                    else:
                        flag = False
                        for i in range(max_hospital_size + 1):
                             if (self.ambulance[0] + 2*value[0], self.ambulance[1] + 2*value[1], i) in self.hospitals:
                                flag = True
                                hospitals_loc.discard((self.ambulance[0] + 2*value[0], self.ambulance[1] + 2*value[1], i))
                                if i > 1:
                                    hospitals_loc.add((self.ambulance[0] + 2*value[0], self.ambulance[1] + 2*value[1], i-1))
                                break
                        patients_loc.discard((self.ambulance[0] + value[0], self.ambulance[1] + value[1]))
                        if not flag:
                            patients_loc.add((self.ambulance[0] + 2*value[0], self.ambulance[1] + 2*value[1])) 
                        next_states.append(State(ambulance_loc, patients_loc, hospitals_loc, path, new_g))
                else:
                    next_states.append(State(ambulance_loc, patients_loc, hospitals_loc, path, new_g))                        
        return next_states
    def eval_h(self, method):
        h = 0
        for p in self.patients:
            min_dis_patients_hospitals = 100000
            for hospital in self.hospitals:
                if method == "Euclidean":
                    dis = (abs(hospital[0] - p[0])**2 + abs(hospital[1] - p[1])**2)**0.5
                elif method == "Manhattan":
                    dis = abs(hospital[0] - p[0]) + abs(hospital[1] - p[1])
                if min_dis_patients_hospitals > dis:
                    min_dis_patients_hospitals = dis
            h += min_dis_patients_hospitals
        return h

## Search Algorithms

- ### BFS
**Breadth-first search** is an algorithm which travers all nodes in a certain hieght and then moves on to the nodes at the next depth level. The key point to reach enough spead is to prevent visiting repeated states. In this implementaion, an ordered set is used to remember unique inserted elements by order. While the set is not empty, the first inserted element will pop and its feasible adjacent states will add to ordered set.

In [5]:
def BFS(initial_state, directions, walls_loc, max_hospital_size):
    unique_states = OrderedSet([initial_state])
    i = 0
    num_all_states = 0
    while True:
        new_states = unique_states[i].get_next_states(directions, walls_loc, max_hospital_size)
        num_all_states += len(new_states)
        for s in new_states:
            if not s.patients:
                return s.path, num_all_states, len(unique_states)
        unique_states.update(new_states)
        i += 1

- ### IDS
**Iterative Deepening Search** is a combination of two algorithms, Breadth-first search (BFS) and Depth-first search (DFS). In this algorithm, a counter (*limit*) which shows the maximum depth to search is count up. At each level, all nodes with depth less than the limit will traverse using `DLS()` which is a recursive implementation of DFS algorithm with limited depth.

In [6]:
def DLS(curr_state, visited_states, limit, directions, walls_loc, max_hospital_size):
    global num_all_states
    if len(curr_state.patients) == 0:
        return curr_state.path
    if limit <= 0:
        return False
    new_states = curr_state.get_next_states(directions, walls_loc, max_hospital_size)
    num_all_states += len(new_states)
    for s in new_states:
        if s in visited_states and visited_states[s] <= len(s.path.split()):
            continue
        else:
            visited_states[s] = len(s.path.split())
        path = DLS(s, visited_states, limit-1, directions, walls_loc, max_hospital_size)
        if path:
            return path
    return False          
    
def IDS(initial_state, directions, walls_loc, max_hospital_size):
    i = 0
    global num_all_states
    num_all_states = 0
    while True:
        visited_states = {initial_state: 0}
        path = DLS(initial_state, visited_states, i, directions, walls_loc, max_hospital_size)
        if path:
            return path, num_all_states, len(visited_states)
        i += 1

- ### A*
It's an informed search strategy that uses an evaluation function to rank nodes and select the most promising for expansion. Evaluation function consists of 2 value, the cost to reach that state (g) and estimation of the cost of reaching the goal called **heuristic** (h). Using `eval_h` heuristic of each state is calculated and stored in its class. At the selection point, the node with minimum g + h is selected from all nodes in the frontier set. Again using `get_next_states()`, the feasible adjacent states will add to the frontier set. If a new state has been already existed and has more g value it should be replaced with the new one. Since a set doesn't return its elements in O(1) access time, it is replaced with a dictionary. The keys of the dictionary are states and the values are g value of them. It should be considered that A* algorithm is optimal if the heuristic is admissible and and search algorithm ignores repeated states.
There are two admissible method to evaluate heuritic, **Manhattan distance** and **Euclidean distance**. Both following heuristics are admissible becuase $h(n) \leq h*(n)$.
    - **Manhattan distance** or **L1 distance** is the sum of the lengths of the projections of the line segment between the points onto the coordinate axes.
    \begin{equation*}
      d_1(p, q) = ||p-q||_1 = \sum_{i=1}^{n}|p_i - q_i| 
    \end{equation*}
    - **Euclidean distance** or **L2 distance** is the ordinary straight-line distance between two points in Euclidean space.
    \begin{equation*}
      d_2(p, q) = ||p-q||_2 = \sum_{i=1}^{n}|p_i - q_i|^2 
    \end{equation*}

In [7]:
def A_star(initial_state, directions, walls_loc, max_hospital_size, method = "Manhattan"):
    initial_state.h = initial_state.eval_h(method)
    visited_states = set()
    frontier_states = {initial_state: 0}
    num_all_states = 0
    while frontier_states:
        selected_state = min(frontier_states, key=lambda k: k.h + k.g)
        if not selected_state.patients:
            return selected_state.path, num_all_states, len(visited_states) + len(frontier_states)
        frontier_states.pop(selected_state)
        visited_states.add(selected_state)
        new_states = selected_state.get_next_states(directions, walls_loc, max_hospital_size)
        num_all_states += len(new_states)
        for s in new_states:
            if s in visited_states:
                continue
            if s in frontier_states:
                if s.g >= frontier_states[s]:
                    continue
                else:
                    frontier_states.pop(s)
            s.h = s.eval_h(method)
            frontier_states[s] = len(s.path.split())

## Result and Conclusion
In our problem initial states consist of the primary location of ambulance, patients and hospitals, actions are feasible movement of ambulance to up, down, left and right, and target state is the one which there is no patients in the map.

In [8]:
def print_results(filename, methods, directions, max_hospital_size = 3, num_of_repeat_methods = 3):
    ambulance_loc, walls_loc, patients_loc, hospitals_loc = read_map(filename)
    initial_state = State(ambulance_loc, patients_loc, hospitals_loc)
    table = "|Method|Distance|Number of All States|Number of Unique States|Average Time|\n|---|---|---|---|---|\n"
    for i, method in enumerate(methods):
        start_time = time.time()
        if method[0] == A_star:
            for _ in range(3):
                path, num_all_states, num_unique_states = method[0](initial_state, directions, walls_loc, max_hospital_size, method[1])
            average_times = (time.time() - start_time)/3
            table += "|" + method[0].__name__ + method[1] + "|" + str(len(path.split())) + "|" + str(num_all_states) + "|" + str(num_unique_states) + "|" + str(average_times) + "|\n"
        else:
            for _ in range(3):
                path, num_all_states, num_unique_states = method[0](initial_state, directions, walls_loc, max_hospital_size)
            average_times = (time.time() - start_time)/3
            table += "|" + method[0].__name__ + "|" + str(len(path.split())) + "|" + str(num_all_states) + "|" + str(num_unique_states) + "|" + str(average_times) + "|\n"  
    table += "<caption style=\"text-align:center\">" + filename + "</caption>\n"
    display(Markdown(table))

In [9]:
directions = {
    "Up": (-1, 0),
    "Left": (0, -1),
    "Down": (1, 0),
    "Right": (0, 1)
}
methods = [[BFS], [IDS], [A_star, "Manhattan"], [A_star, "Euclidean"]]
for i in range(3):
    print_results("testcases/test" + str(i + 1) + ".txt", methods, directions)

|Method|Distance|Number of All States|Number of Unique States|Average Time|
|---|---|---|---|---|
|BFS|11|939|447|0.0054314931233723955|
|IDS|11|4092|423|0.01670527458190918|
|A_starManhattan|11|433|210|0.0025680859883626304|
|A_starEuclidean|11|468|222|0.0029081503550211587|
<caption style="text-align:center">testcases/test1.txt</caption>


|Method|Distance|Number of All States|Number of Unique States|Average Time|
|---|---|---|---|---|
|BFS|27|24756|10468|0.13984338442484537|
|IDS|27|472421|3563|2.3853904406229653|
|A_starManhattan|27|12480|5608|0.3984987735748291|
|A_starEuclidean|27|13736|6118|0.47394657135009766|
<caption style="text-align:center">testcases/test2.txt</caption>


|Method|Distance|Number of All States|Number of Unique States|Average Time|
|---|---|---|---|---|
|BFS|39|80333|31750|0.4358649253845215|
|IDS|39|2154235|8291|12.141573905944824|
|A_starManhattan|39|31782|13303|1.564466158548991|
|A_starEuclidean|39|37072|15363|2.052085796991984|
<caption style="text-align:center">testcases/test3.txt</caption>


As we would expect all 4 methods are optimal and they found the shortest path to solve the problem. If we compare 4 methods based on speed, A* methods and BFS are faster than IDS, but as you see IDS stores much fewer states.It should be noted that to increase the speed, we stores all unique visited states in IDS but originally only O(m) nodes should be stores in this method. There is a table below to compare these methods:

|Method|complete|optimal|time|space|
|---|---|---|---|---|
|BFS|yes|yes|O$(b^{d})$|O$(b^{d})$|
|Original IDS|yes|yes|O$(b^{d})$|O$(bd)$| 
|$A^*$|yes|yes|$exp$|$exp$|
<caption  style="text-align:center">Properties of different algorithms</caption>